In [7]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit, RepeatedKFold
import numpy as np
from itertools import combinations
import statsmodels.api as sm
import statsmodels.tools as st
from statsmodels.stats.outliers_influence import variance_inflation_factor
import time
import matplotlib.pyplot as plt
import pprint
import statistics
import time
%matplotlib inline

In [15]:
# response = 'rheomix final deg time min'
response = 'rheomix stability time min'

redo = 5
rorder = 1

In [16]:
file1 = 'Total_model_subset.csv'
df_model  = pd.read_csv(file1)

In [17]:
def import_data(experimental_file, response_file, response):
    'imports data'
    'experimental_file = experimental design csv filename i.e experimental.csv'
    'response_file = results csv filename i.e Response.csv'
    'response = reponse name: i.e rheomix final deg time min or rheomix stability time min'
    
    experimental_df = pd.read_csv(experimental_file)
    response_df = pd.read_csv(response_file)
    
    X = experimental_df[experimental_df.columns.values.tolist()[1:]].values
    y = response_df[response].values
    max1 = max(y)
    min1 = min(y)

    y_norm = [2*((i-min1)/(max1-min1)) - 1 for i in y]
    
    
    X_linear = X
    linear_terms = experimental_df.columns.values.tolist()[1:]
    
    return y_norm, X_linear, linear_terms, experimental_df, response_df

def X_gen(model, X_linear):
    
    for i, j in enumerate(model):
        
        if i == 0 and len(j) == 2: 
            X_new = X_linear[:, model[0][1]]
        
        else:
            
            if len(j) == 2:
                add = X_linear[:, j[1]]

            if len(j) == 3:
                add = X_linear[:, j[1]]*X_linear[:, j[2]]

            X_new2 = np.column_stack((X_new, add))
            X_new = X_new2

    
    return X_new

In [18]:
dict1 = {'Xpvc': 0, 'Xfiller': 1, 'Xfr': 2, 'Xstabiliser': 3, 'Xdinp': 4, 'Xldh':5, "Xsph": 6}
list_k = ['k3','k5','k10','k15','k20','LOO']
list_k_no = [3,5,10,15,20,46]

# list_k = ['k3']
# list_k_no = [3]

# list_k = ['LOO']
# list_k_no = [46]


for K_name, K_num in zip(list_k, list_k_no):
    
    start = time.time()
    list_kscore = [-1000]*28
    list_models = ["test"]*28
    list_noTerms = a = np.linspace(1,28,28)

    experimental_file = 'experimental.csv'
    response_file = 'Response.csv'

    y, X_linear, linear_terms, experimental_df, response_df = import_data(experimental_file, response_file, response)
    y1 = np.array(y)

    models_list = np.array(df_model['Model'].copy())
    no_terms_list = np.array(df_model['No_terms'].copy())
    
    if K_name == 'LOO':
        redo1 = 1
    else:
        redo1 = redo

    for model, no_terms in zip(models_list, no_terms_list):

        cnt = 0                  
        a = model.split('--')[1:-1]
        model_fin = []
        for b in a:
            if len(b.split('*')) == 1:
                model_fin.append([b, dict1[b]])
            else:
                model_fin.append([b, dict1[b.split('*')[0]], dict1[b.split('*')[1]]])

        X = X_gen(model_fin, X_linear)
        
            
        if len(model_fin) == 1:

            X1 = X.reshape(len(X), 1)
            model_obj = LinearRegression(fit_intercept=False)
            my_cv = RepeatedKFold(n_splits=K_num, random_state= rorder, n_repeats = redo1)

            score = cross_val_score(model_obj, X1, y1, cv=my_cv, scoring='neg_mean_squared_error')
            av_score = sum(score)/len(score)

        else:

            model_obj = LinearRegression(fit_intercept=False)
            my_cv = RepeatedKFold(n_splits=K_num, random_state= rorder, n_repeats = redo1)

            score = cross_val_score(model_obj, X, y1, cv=my_cv, scoring='neg_mean_squared_error')
            av_score = sum(score)/len(score)
            
        if av_score > list_kscore[no_terms-1]:
            list_kscore[no_terms-1] = av_score
            list_models[no_terms-1] = model

    df_results = pd.DataFrame(columns=['Model','No_terms', K_name])
    df_results['Model'] = list_models
    df_results['No_terms'] = list_noTerms
    df_results[K_name] = list_kscore
    df_results.to_csv('Total_model_subset_{}_Kfold_{}.csv'.format(response, K_name))
    end = time.time()
    print(K_name, ' : ', end - start)

k3  :  32.3789279460907
k5  :  59.302640199661255
k10  :  115.64906644821167
k15  :  177.27697277069092
k20  :  244.905606508255
LOO  :  118.39533400535583


In [19]:
df_results

,Model,No_terms,LOO
0,--Xpvc--,1.0,-0.215485
1,--Xpvc--Xfiller--,2.0,-0.188715
2,--Xdinp--Xldh--Xdinp*Xldh--,3.0,-0.116594
3,--Xfiller--Xdinp--Xldh--Xdinp*Xldh--,4.0,-0.107033
4,--Xfiller--Xstabiliser--Xdinp--Xldh--Xdinp*Xldh--,5.0,-0.108590
5,--Xfiller--Xstabiliser--Xdinp--Xldh--Xstabilis...,6.0,-0.100255
6,--Xpvc--Xfiller--Xstabiliser--Xdinp--Xldh--Xst...,7.0,-0.085530
7,--Xpvc--Xfiller--Xstabiliser--Xdinp--Xldh--Xfi...,8.0,-0.085462
8,--Xpvc--Xfiller--Xstabiliser--Xdinp--Xldh--Xpv...,9.0,-0.086658
9,--Xpvc--Xfr--Xstabiliser--Xdinp--Xldh--Xsph--X...,10.0,-0.087970
